In [ ]:
# Calling important modules:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import pandas_datareader as pdr
import yfinance as yf
import datetime
import bs4 as bs


In [ ]:
# Download tickers
resp = requests.get('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
soup = bs.BeautifulSoup(resp.text, 'lxml')
table = soup.find('table', {'class': 'wikitable sortable'})


tickers = []

for row in table.findAll('tr')[1:]:
    ticker = row.findAll('td')[0].text
    tickers.append(ticker)

tickers = [s.replace('\n', '') for s in tickers]



In [ ]:
# Define date and download

start = datetime.datetime(2020, 1, 1)
end = datetime.datetime(2023, 11, 30)
data = yf.download(tickers, start=start, end=end)

In [ ]:
# Stacking data

df = data.stack().reset_index().rename(index=str, columns={"level_1": "Symbol"}).sort_values(['Symbol','Date'])
df.set_index('Date', inplace=True)
df.head()

In [ ]:
# Calculation of return and f return monthly

df['ret'] = df['Adj Close'].pct_change()
df['f_ret'] = df['ret'].shift(-1)
df = df.groupby("Symbol").resample("M").last()
df.head()

In [ ]:
# Calculation market

mkt = yf.download(tickers='^GSPC', start="2020-01-01", end='2023-11-30')
mkt = mkt.resample('M').last()
mkt['mkt'] = mkt['Adj Close'].pct_change()
mkt['f_mkt'] = mkt['mkt'].shift(-1)
mkt = mkt.reset_index()[['Date', 'mkt', 'f_mkt']]
mkt.head()

Ris-free return of Market

In [ ]:
# Calculation of risk-free rate

rf = pdr.DataReader('DGS3MO', 'fred', start="1900-1-1", end="2023-12-31").resample('M').last()
rf['DGS3MO'] = rf['DGS3MO']/1200
rf['rf'] = rf['DGS3MO'].shift()
rf = rf.reset_index()[['DATE', 'DGS3MO', 'rf']].rename(columns={'DATE':'Date', 'DGS3MO':'f_rf'})
rf.head()

In [ ]:
# Merging data

df = pd.merge(df, mkt, on='Date') # Inner merge because missing values will drop from regressions
df = pd.merge(df, rf, on='Date')
df = df.sort_values(['Symbol', 'Date'])
# col = ["mkt_y", "f_mkt_y", "f_rf_y", "rf_y"]
# df.drop(col, axis =1)
# df["mkt_x"].rename()
df.head()

In [ ]:
# define excess returns
df['ret-rf'] = df['ret'] - df['rf']
df['mkt-rf'] = df['mkt'] - df['rf']
df.head()

In [ ]:
df.describe().to_excel("df Statistics.xlsx")

In [ ]:
# Covariance and expected return calculation

print(len((df.groupby("Symbol"))))
er = []
var = []
cov = np.array([df.groupby("Symbol")[["ret"]].cov()["ret"]])


for i in range(0,len(df.groupby("Symbol"))):

    er.append(df.groupby("Symbol")["ret"].mean()[i])
    var.append(df.groupby("Symbol")["ret"].var()[i])


# print(er)
#################
covariance = df.groupby("Symbol")[["ret"]].cov()["ret"]
for i in range(0,500):
    # for j in range(0,500):
    cov =  np.insert(cov, i, covariance, axis=0)

for i in range(0,501):
    cov[i,i] = var[i]



In [ ]:
# For example we choose 2 symbols:

# Define expected returns and covariance matrix
# R_f = df.groupby("Symbol")["ret-rf"].mean()[0]
R_f = 0.03
E_R_A = er[0]
E_R_B = er[1]
R = np.array([E_R_A, E_R_B])

sigma2_A = var[0]
sigma2_B = var[1]
sigma_A_B = cov[0][1]
Sigma = np.array([[sigma2_A, sigma_A_B], [sigma_A_B, sigma2_B]])

# Calculate expected returns and variances for different portfolio weights allowing for short selling
weights_A = 0.5  # Allowing weights to be negative and exceed 1
weights_B = 1 - weights_A

E_R_p = weights_A * E_R_A + weights_B * E_R_B
std_p = np.sqrt(weights_A**2 * sigma2_A + weights_B**2 * sigma2_B + 2 * weights_A * weights_B * sigma_A_B)


In [ ]:
# Efficiency of 2 assets
E_R_p = weights_A * E_R_A + weights_B * E_R_B
std_p = np.sqrt(weights_A**2 * sigma2_A + weights_B**2 * sigma2_B + 2 * weights_A * weights_B * sigma_A_B)

# Plotting the expected returns against the variances with short selling
plt.figure(figsize=(10, 6))
plt.plot(std_p, E_R_p, c='blue', linewidth=2, label='Efficient Frontier with Only Risky Assets')
plt.title('Efficient Frontier with A Risk-Free Asset')
plt.xlabel('Portfolio Standard Deviation')
plt.ylabel('Portfolio Expected Return')

# Find the tangency portfolio
w_t = np.dot(np.linalg.inv(Sigma), R - R_f)
w_t = w_t/np.sum(w_t)

# Calculate the expected return and standard deviation of the portfolio
E_R_t = np.sum(w_t*R)
std_t = np.sqrt(np.dot(w_t.T, np.dot(Sigma, w_t)))

# Plot the portfolio
plt.scatter(std_t, E_R_t, c='red', marker='*', s=200, label='The Tangency Portfolio')

# Plot all the portfolios with the risk-free asset
w_f = np.linspace(-3, 3, 1000)
E_R_p = w_f * R_f + (1-w_f) * E_R_t
std_p = (1-w_f) * std_t
plt.plot(std_p, E_R_p, c='red', linewidth=2, label='Efficient Frontier with Risk-Free Asset')

# Plot the risk-free asset
plt.scatter(0, R_f, c='orange', marker='^')


# plt.xlim([0, 0.8])
# plt.ylim([0, 0.20])
plt.legend(loc='upper left')
plt.grid(True)
plt.show()

In [ ]:
# Efficiency of 2 assets with and without selling

weights_A = np.linspace(-1, 2, 300)  # Allowing weights to be negative and exceed 1
weights_B = 1 - weights_A

E_R_p = weights_A * E_R_A + weights_B * E_R_B
std_p = np.sqrt(weights_A**2 * sigma2_A + weights_B**2 * sigma2_B + 2 * weights_A * weights_B * sigma_A_B)

# Plotting the expected returns against the variances with short selling
plt.figure(figsize=(10, 6))
plt.scatter(std_p, E_R_p, c='blue', marker='o', s=5, label='Portfolios with Short Selling')
plt.scatter(np.sqrt(sigma2_A), E_R_A, c='red', marker='*', s=200, label='100% Asset A')
plt.scatter(np.sqrt(sigma2_B), E_R_B, c='yellow', marker='*', s=200, label='100% Asset B')
plt.title('Efficient Frontier with and without Short Selling')
plt.xlabel('Portfolio Standard Deviation')
plt.ylabel('Portfolio Expected Return')
plt.legend(loc='upper left')
plt.grid(True)
plt.show()

In [ ]:
# Plotting the expected returns against the variances with highlighted portfolios
plt.figure(figsize=(10, 6))
plt.scatter(std_p, E_R_p, c='blue', marker='o', s=5, label='Portfolios')
plt.scatter(np.sqrt(sigma2_A), E_R_A, c='red', marker='*', s=200, label='100% Asset A')
plt.scatter(np.sqrt(sigma2_B), E_R_B, c='orange', marker='*', s=200, label='100% Asset B')
plt.title('Efficient Frontier for Two-Asset Portfolio')
plt.xlabel('Portfolio Standard Deviation')
plt.ylabel('Portfolio Expected Return')
plt.legend(loc='upper left')
plt.grid(True)
plt.show()

In [ ]:
E_R_A = er[0]
E_R_B = er[1]
E_R_C = er[2]
R = np.array([E_R_A, E_R_B, E_R_C])

sigma2_A = var[0]
sigma2_B = var[1]
sigma2_C = var[2]
sigma_A_B = cov[0][1]
sigma_A_C = cov[0][2]
sigma_B_C = cov[2][3]
Sigma = np.array([[sigma2_A, sigma_A_B, sigma_A_C], [sigma_A_B, sigma2_B, sigma_B_C], [sigma_A_C, sigma_B_C, sigma2_C]])

# Generate N x N portfolios
N = 100
num_portfolios = N*N
E_R_p = np.zeros((N, N))
std_p = np.zeros((N, N))

w = np.linspace(-4, 4, N)
for i, w_A in enumerate(w):
    for j, w_B in enumerate(w):
        w_C = 1.0 - w_A - w_B
        weights = np.array([w_A, w_B, w_C])
        portfolio_return = np.sum(weights * R)
        portfolio_stddev = np.sqrt(np.dot(weights.T, np.dot(Sigma, weights)))
        E_R_p[i,j] = portfolio_return
        std_p[i,j] = portfolio_stddev

# Plot the results
plt.figure(figsize=(10, 6))
plt.scatter(std_p, E_R_p, c='cyan', s=1, marker='o', label='Random Portfolios')
plt.scatter(np.sqrt(sigma2_A), E_R_A, c='red', marker='*', s=100, label='Asset A')
plt.scatter(np.sqrt(sigma2_B), E_R_B, c='green', marker='*', s=100, label='Asset B')
plt.scatter(np.sqrt(sigma2_C), E_R_C, c='yellow', marker='*', s=100, label='Asset C')
plt.title('Efficient Frontier with Three Assets')
plt.xlabel('Portfolio Standard Deviation')
plt.ylabel('Portfolio Expected Return')
plt.legend(loc='upper left')
# plt.xlim([0,0.9])
# plt.ylim([0.02,0.18])

plt.grid(True)
plt.show()

In [ ]:
def estimate_mu_sigma_rf(df):
    assets = df.columns.drop("rf")
    mu = df["Adj Close"].mean(axis=0)
    Sigma = df["Adj Close"].cov(other=df["Adj Close"])
    rf = df['rf'].mean()
    return mu, Sigma, rf

# The function to calculate the tangency portfolio
def find_tangency_is(mu, Sigma, rf):
    w_t = np.dot(np.linalg.inv(Sigma), mu - rf)
    w_t = w_t/np.sum(w_t)
    return w_t

estimate_mu_sigma_rf(df)

In [ ]:
def find_tangency_oos(df, window=60, method="expanding"):

    # Get the list of assets
    assets = df.columns.drop("rf")

    # Initiate the time-series of the tangency portfolio
    weights = pd.DataFrame(index=df.index, columns=assets, data=np.nan)
    returns = pd.Series(index=df.index, data=np.nan)

    for i in range(window, len(df)):

        # Select the estimation period
        if method == "expanding":
            df_est = df.iloc[:i-1]
        elif method == "rolling":
            df_est = df.iloc[i-window:i-1]

        # Find the weights of the tangency portfolio
        mu, Sigma, _ = estimate_mu_sigma_rf(df_est)
        rf = df.iloc[i]["rf"]

        # Find the tangency portfolio
        w_t = find_tangency_is(mu, Sigma, rf)

        # Save the weights
        weights.iloc[i] = w_t

        # Find the return of the tangency portfolio over the next period
        returns.iloc[i] = (w_t * df[assets].iloc[i]).sum()

    # Calculate the Sharpe ratio of the tangency portfolio
    SR = (returns - df["rf"]).mean() / returns.std()

    return returns, weights, SR

ret_expanding, weights_expanding, SR_expanding = find_tangency_oos(df)
ret_rolling, weights_rolling, SR_rolling = find_tangency_oos(df, method="rolling")

In [ ]:
# Number of simulations

mu, Sigma, rf = estimate_mu_sigma_rf(df)
# mu, Sigma, rf
num_portfolios = 10000

# Initialize arrays to store portfolio weights, returns, volatilities, and Sharpe ratios
portfolio_weights = np.zeros((num_portfolios, len(tickers)))
portfolio_returns = np.zeros(num_portfolios)
portfolio_stds = np.zeros(num_portfolios)
sharpe_ratios = np.zeros(num_portfolios)

# Randomly assign weights to assets and normalize
portfolio_weights[:, :2] = -5 + 10*np.random.random(size=(num_portfolios,2))
portfolio_weights[:, 2] = 1 - np.sum(portfolio_weights[:, :2], axis=1)

# Calculate expected portfolio return and volatility
for i in range(num_portfolios):
    portfolio_returns[i] = np.dot(portfolio_weights[i], mu)
    portfolio_stds[i] = np.sqrt(np.dot(portfolio_weights[i].T, np.dot(Sigma, portfolio_weights[i])))

# Calculate Sharpe ratio
sharpe_ratios = (portfolio_returns - rf) / portfolio_stds

# Plot histogram of Sharpe ratios
plt.figure(figsize=(10, 6))
plt.hist(sharpe_ratios, bins=50, color='blue', alpha=0.7)
plt.axvline(SR_t, color='red', linestyle='--', linewidth=3, label='Tangency Portfolio')
plt.axvline(SR_ev, color='orange', linestyle=':', linewidth=3, label='Equally-Weighted Portfolio')

plt.title('Histogram of Sharpe Ratios of Simulated Portfolios')
plt.xlabel('Sharpe Ratio')
plt.ylabel('Frequency')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# Plot histogram of Sharpe ratios
plt.figure(figsize=(10, 6))
plt.hist(sharpe_ratios, bins=50, color='blue', alpha=0.7)
plt.axvline(SR_ev, color='orange', linestyle=':', linewidth=3, label='Equally-Weighted Portfolio')
plt.axvline(SR_expanding, color='brown', linestyle='-.', linewidth=3, label='Expanding Window Tangency Portfolio')
plt.axvline(SR_rolling, color='black', linestyle='-', linewidth=3, label='Rolling Window Tangency Portfolio')

plt.title('Histogram of Sharpe Ratios of Simulated Portfolios')
plt.xlabel('Sharpe Ratio')
plt.ylabel('Frequency')
plt.legend(loc='upper left')
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
def find_tangency_oos(df, window=60, method="expanding", estimate_Sigma=True):

    # Get the list of assets
    assets = df.columns.drop("rf")

    # Initiate the time-series of the tangency portfolio
    weights = pd.DataFrame(index=df.index, columns=assets, data=np.nan)
    returns = pd.Series(index=df.index, data=np.nan)

    for i in range(window, len(df)):

        # Select the estimation period
        if method == "expanding":
            df_est = df.iloc[:i-1]
        elif method == "rolling":
            df_est = df.iloc[i-window:i-1]

        # Find the weights of the tangency portfolio
        mu, Sigma, _ = estimate_mu_sigma_rf(df_est)
        rf = df.iloc[i]["rf"]

        if not estimate_Sigma:
            Sigma = np.eye(len(mu))

        # Find the tangency portfolio
        w_t = find_tangency_is(mu, Sigma, rf)

        # Save the weights
        weights.iloc[i] = w_t

        # Find the return of the tangency portfolio over the next period
        returns.iloc[i] = (w_t * df[assets].iloc[i]).sum()

    # Calculate the Sharpe ratio of the tangency portfolio
    SR = (returns - df["rf"]).mean() / returns.std()

    return returns, weights, SR


In [ ]:
SR_rolling
# Plot histogram of Sharpe ratios
plt.figure(figsize=(10, 6))
plt.hist(sharpe_ratios, bins=50, color='blue', alpha=0.7)
plt.axvline(SR_ev, color='orange', linestyle=':', linewidth=3, label='Equally-Weighted Portfolio')
plt.axvline(SR_expanding, color='brown', linestyle='-.', linewidth=3, label='Expanding Window Tangency Portfolio')
plt.axvline(SR_rolling, color='black', linestyle='-', linewidth=3, label='Rolling Window Tangency Portfolio')

plt.title('Histogram of Sharpe Ratios of Simulated Portfolios')
plt.xlabel('Sharpe Ratio')
plt.ylabel('Frequency')
plt.legend(loc='upper left')
plt.grid(True)
plt.tight_layout()
plt.show()